# Synthesizing data with time-sensitive location data

## Reconstructing books/members/events

In [1]:
import sys; sys.path.insert(0,'..')
from geotaste import *

In [2]:
def get_book_id(x): return x.split('/books/',1)[1][:-1] if '/books/' in x else ''

@cache
def get_books_df(): 
    df=get_urlpath_df('books').fillna('')
    df['book_id']=df.uri.apply(get_book_id)
    return df.set_index('book_id')

In [3]:
df_books=get_books_df()
# df_books

In [4]:
def get_member_id(x): return x.split('/members/',1)[1][:-1] if '/members/' in x else ''

@cache
def get_members_df(): 
    df=get_urlpath_df('members').fillna('')
    df['is_expat'] = df['nationalities'].apply(lambda x: 'France' not in x)
    df['has_wikipedia'] = df['wikipedia_url']!=''
    df['has_viaf'] = df['viaf_url']!=''
    df['birth_decade'] = [str(x)[:3]+'0s' if x else '' for x in df['birth_year']]
    df['generation'] = df['birth_year'].apply(parse_generation)
    df['member_id']=df.uri.apply(get_member_id)
    return df.set_index('member_id')

In [5]:
df_members = get_members_df()
# df_members

In [6]:
def get_event_id(row):
    base=f'{row.member_id} {(row.event_type+"s").upper()} {row.book_id}'
    if not row.start_date: return base
    if row.end_date and row.start_date!=row.end_date:
        return f'{base} FROM {row.start_date} TO {row.end_date}'
    return f'{base} ON {row.start_date}'

@cache
def get_events_df(): 
    # get as exists
    df = get_urlpath_df('events').fillna('')
    
    ## expand by semi colon!
    def getrowinfo(row, i):
        odx={}
        for k,v in dict(row).items():
            if ';' in str(v):
                vs = str(v).split(';')
                v=vs[i]
            odx[k]=v
        return odx
    
    df_events_expanded = pd.DataFrame(
        {**getrowinfo(row,mi), 'member_uri':muri, 'member_id':get_member_id(muri), 'book_id':get_book_id(row.item_uri)}
        for i,row in df.iterrows()
        for mi,muri in enumerate(row.member_uris.split(';'))
    )
    
    odf = df_events_expanded[['member_id','book_id']+[col for col in df if not col.split('_')[0] in {'member','item'}]]
    odf['event_id'] = odf.apply(get_event_id,axis=1)
    return odf.set_index('event_id')


In [7]:
df_events = get_events_df()
df_events

,member_id,book_id,event_type,start_date,end_date,subscription_price_paid,subscription_deposit,subscription_duration,subscription_duration_days,subscription_volumes,...,subscription_purchase_date,reimbursement_refund,borrow_status,borrow_duration_days,purchase_price,currency,source_type,source_citation,source_manifest,source_image
event_id,,,,,,,,,,,,,,,,,,,,,
linossier-raymonde GENERICS butler-pigs-pigs ON 1920,linossier-raymonde,butler-pigs-pigs,Generic,1920,,,,,,,...,,,,,,,Lending Library Card,"Sylvia Beach, Raymonde Linossier Lending Libra...",https://figgy.princeton.edu/concern/scanned_re...,https://iiif.princeton.edu/loris/figgy_prod/00...
garreta SUBSCRIPTIONS ON 1921,garreta,,Subscription,1921,,,,,,,...,1921,,,,,FRF,Address Book,"Sylvia Beach, Address Book 1919–1935, box 69, ...",,
rhys BORROWS conrad-typhoon FROM 1922 TO 1922-08-23,rhys,conrad-typhoon,Borrow,1922,1922-08-23,,,,,,...,,,Returned,,,,Lending Library Card,"Sylvia Beach, Rhys Lending Library Card, Box 4...",https://figgy.princeton.edu/concern/scanned_re...,https://iiif.princeton.edu/loris/figgy_prod/67...
walsh GENERICS bennett-pretty-lady ON 1922,walsh,bennett-pretty-lady,Generic,1922,,,,,,,...,,,,,,,Lending Library Card,"Sylvia Beach, Ernest Walsh Lending Library Car...",https://figgy.princeton.edu/concern/scanned_re...,https://iiif.princeton.edu/loris/figgy_prod/af...
lincoln-1 SUBSCRIPTIONS ON 1922,lincoln-1,,Subscription,1922,,,7.0,,,,...,1922,,,,,FRF,Address Book,"Sylvia Beach, Address Book 1919–1935, box 69, ...",,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
gutmann-henri REIMBURSEMENTS,gutmann-henri,,Reimbursement,,,,,,,,...,,,,,,,Address Book,"Sylvia Beach, Address Book 1919–1935, box 69, ...",,
bernheim-antoinette GENERICS isherwood-lions-shadows-education,bernheim-antoinette,isherwood-lions-shadows-education,Generic,,,,,,,,...,,,,,,,Lending Library Card,"Sylvia Beach, Antoinette Bernheim Lending Libr...",https://figgy.princeton.edu/concern/scanned_re...,https://iiif-cloud.princeton.edu/iiif/2/eb%2F7...
baldwin-mr REIMBURSEMENTS,baldwin-mr,,Reimbursement,,,,,,,,...,,,,,,,Address Book,"Sylvia Beach, Address Book 1919–1935, box 69, ...",,


## Finding locations

In [56]:
def get_dwelling_id(row):
    o=f'{row.member_id} DWELT AT {row.street_address.replace(" ","-")}'
    if row.start_date: o+=f' FROM {row.start_date}'
    if row.end_date: o+=f' UNTIL {row.end_date}'
    return o
    
@cache
def get_dwellings_df(): 
    df=get_urlpath_df('dwellings').fillna('')
    startcols = ['member_id','start_date','end_date']
    endcols = [c for c in df if c.endswith('_id')]
    df['member_id']=df.member_uri.apply(get_member_id)
    odf=df[startcols + [c for c in df if c not in set(startcols+endcols)] + endcols]
    odf['dwelling_id'] = odf.apply(get_dwelling_id, axis=1)
    return odf.set_index('dwelling_id')
    

In [59]:
df_dwellings = get_dwellings_df()
# df_dwellings

In [60]:
# Number of start dates?
len(df_dwellings[df_dwellings.start_date!=''])

49

In [61]:
# Number of end dates?
len(df_dwellings[df_dwellings.end_date!=''])

46

In [62]:
df_dwellings[df_dwellings.start_date!=''].head(2)

,member_id,start_date,end_date,member_uri,start_date_precision,end_date_precision,street_address,city,postal_code,latitude,longitude,arrrondissement,person_id,account_id,address_id,location_id,care_of_person_id,country_id
dwelling_id,,,,,,,,,,,,,,,,,,
barlett-jessie DWELT AT 26-rue-de-Lübeck FROM 1924-01-01,barlett-jessie,1924-01-01,,https://shakespeareandco.princeton.edu/members...,DatePrecision(year|month),,26 rue de Lübeck,Paris,75116,48.86522,2.29188,16.0,2289,1048,1359,970,,2.0
barlett-jessie DWELT AT 14-rue-d'Assas FROM 1923-09-01 UNTIL 1923-12-01,barlett-jessie,1923-09-01,1923-12-01,https://shakespeareandco.princeton.edu/members...,DatePrecision(year|month),DatePrecision(year|month),14 rue d'Assas,Paris,75006,48.84928,2.32841,6.0,2289,1048,3626,852,,2.0


## New combined df

In [186]:
@cache
def get_borrow_events_df():
    df = get_events_df().reset_index()
    df = df[df.event_type=='Borrow']
    df = df[[c for c in df if c.split('_')[0] not in {'subscription','reimbursement','purchase','currency'} and c not in {'event_type'}]]
    df['dwelling_ids'] = df.apply(lambda row: find_dwelling_id(row, verbose=False), axis=1)
    return df.set_index('event_id')
    

In [187]:
def find_dwelling_id(borrow_row, sep='; ', verbose=True):
    memid = borrow_row.member_id
    df = df_dwellings[df_dwellings.member_id==memid]
    
    # if no dwellign at all?
    if not len(df): 
        return ''
    
    # if only one?
    elif len(df)==1: 
        return df.index[0]
    
    # ok, if multiple:
    else:
        # if start date?
        borrow_start = borrow_row.start_date
        borrow_end = borrow_row.end_date
        
        if verbose: 
            print('looking for:',borrow_row.event_id,'\n')
            print('possibilities:',sep.join(df.index),'\n')
        
        if borrow_start:
            # If we know when the borrow began, then exclude dwellings which end before that date
            df = df[df.end_date.apply(lambda x: not (x and x<borrow_start))]
        if borrow_end:
            # If we know when the borrow ended, then exclude dwellings which begin after that date
            df = df[df.start_date.apply(lambda x: not (x and x>borrow_end))]
        
        o=sep.join(df.index)
        if verbose: print('solution:',o,'\n\n')
        return o


In [188]:
df_borrows = get_borrow_events_df()
df_borrows

,member_id,book_id,start_date,end_date,borrow_status,borrow_duration_days,source_type,source_citation,source_manifest,source_image,dwelling_ids
event_id,,,,,,,,,,,
rhys BORROWS conrad-typhoon FROM 1922 TO 1922-08-23,rhys,conrad-typhoon,1922,1922-08-23,Returned,,Lending Library Card,"Sylvia Beach, Rhys Lending Library Card, Box 4...",https://figgy.princeton.edu/concern/scanned_re...,https://iiif.princeton.edu/loris/figgy_prod/67...,
lanux-eyre-de BORROWS woolf-night-day FROM 1923 TO 1923-11-07,lanux-eyre-de,woolf-night-day,1923,1923-11-07,Returned,,Lending Library Card,"Sylvia Beach, Eyre de Lanux Lending Library Ca...",https://figgy.princeton.edu/concern/scanned_re...,https://iiif.princeton.edu/loris/figgy_prod/c5...,lanux-eyre-de DWELT AT 126-boulevard-du-Montpa...
tery BORROWS james-joyce FROM 1924 TO 1924-10-22,tery,james-joyce,1924,1924-10-22,Returned,,Lending Library Card,"Sylvia Beach, Mlle Simone Tery Lending Library...",https://figgy.princeton.edu/concern/scanned_re...,https://iiif.princeton.edu/loris/figgy_prod/c2...,tery DWELT AT 4-rue-de-l'Hôtel-Colbert
tery BORROWS freeman-portrait-george-moore FROM 1924 TO 1924-10-22,tery,freeman-portrait-george-moore,1924,1924-10-22,Returned,,Lending Library Card,"Sylvia Beach, Mlle Simone Tery Lending Library...",https://figgy.princeton.edu/concern/scanned_re...,https://iiif.princeton.edu/loris/figgy_prod/c2...,tery DWELT AT 4-rue-de-l'Hôtel-Colbert
macleish-ada BORROWS stern-tents-israel FROM 1926 TO 1926-01-14,macleish-ada,stern-tents-israel,1926,1926-01-14,Returned,,Lending Library Card,"Sylvia Beach, Ada and Archibald MacLeish Lendi...",https://figgy.princeton.edu/concern/scanned_re...,https://iiif.princeton.edu/loris/figgy_prod/22...,macleish-ada DWELT AT 10-parc-de-Montretout; m...
...,...,...,...,...,...,...,...,...,...,...,...
hommel-bernard BORROWS heine-poems,hommel-bernard,heine-poems,,,Returned,,Lending Library Card,"Sylvia Beach, Bernard Hommel Lending Library C...",https://figgy.princeton.edu/concern/scanned_re...,https://iiif.princeton.edu/loris/figgy_prod/e4...,hommel-bernard DWELT AT 12-rue-de-l'Odéon
hommel-bernard BORROWS cowley-exiles-return-narrative,hommel-bernard,cowley-exiles-return-narrative,,,Returned,,Lending Library Card,"Sylvia Beach, Bernard Hommel Lending Library C...",https://figgy.princeton.edu/concern/scanned_re...,https://iiif.princeton.edu/loris/figgy_prod/e4...,hommel-bernard DWELT AT 12-rue-de-l'Odéon
hommel-bernard BORROWS wilson-axels-castle-study,hommel-bernard,wilson-axels-castle-study,,,Returned,,Lending Library Card,"Sylvia Beach, Bernard Hommel Lending Library C...",https://figgy.princeton.edu/concern/scanned_re...,https://iiif.princeton.edu/loris/figgy_prod/e4...,hommel-bernard DWELT AT 12-rue-de-l'Odéon


In [189]:
df_borrows.iloc[0]

member_id                                                            rhys
book_id                                                    conrad-typhoon
start_date                                                           1922
end_date                                                       1922-08-23
borrow_status                                                    Returned
borrow_duration_days                                                     
source_type                                          Lending Library Card
source_citation         Sylvia Beach, Rhys Lending Library Card, Box 4...
source_manifest         https://figgy.princeton.edu/concern/scanned_re...
source_image            https://iiif.princeton.edu/loris/figgy_prod/67...
dwelling_ids                                                             
Name: rhys BORROWS conrad-typhoon FROM 1922 TO 1922-08-23, dtype: object

In [190]:
# df_dwellings.query('member_id=="rhys"')

In [191]:
# df_members.loc['rhys']

In [192]:
find_dwelling_id(df_borrows.reset_index().iloc[1])

looking for: lanux-eyre-de BORROWS woolf-night-day FROM 1923 TO 1923-11-07 

possibilities: lanux-eyre-de DWELT AT 126-boulevard-du-Montparnasse; lanux-eyre-de DWELT AT 17-rue-Visconti 

solution: lanux-eyre-de DWELT AT 126-boulevard-du-Montparnasse; lanux-eyre-de DWELT AT 17-rue-Visconti 




'lanux-eyre-de DWELT AT 126-boulevard-du-Montparnasse; lanux-eyre-de DWELT AT 17-rue-Visconti'

In [193]:
find_dwelling_id(df_borrows.iloc[2])

"tery DWELT AT 4-rue-de-l'Hôtel-Colbert"

In [194]:
find_dwelling_id(df_borrows.iloc[3])

"tery DWELT AT 4-rue-de-l'Hôtel-Colbert"

In [195]:
find_dwelling_id(df_borrows[df_borrows.member_id=='hemingway-ernest'].reset_index().iloc[0])

looking for: hemingway-ernest BORROWS wilder-bridge-san-luis FROM 1927-11 TO 1927-12-07 

possibilities: hemingway-ernest DWELT AT 112-chemin-des-Mougins FROM 1926-08-02 UNTIL 1926-08-12; hemingway-ernest DWELT AT 113-rue-Notre-Dame-des-Champs FROM 1924-02-08 UNTIL 1926-01-01; hemingway-ernest DWELT AT  FROM 1926-05-14 UNTIL 1926-05-28; hemingway-ernest DWELT AT 69-rue-Froidevaux FROM 1926-08-12 UNTIL 1927-01-01; hemingway-ernest DWELT AT  FROM 1931-12-19 UNTIL 1939-12-24; hemingway-ernest DWELT AT Promenade-10 FROM 1926-12-25 UNTIL 1927-03-10; hemingway-ernest DWELT AT Passeig-de-Gràcia-68 FROM 1937-01-01 UNTIL 1937-01-01; hemingway-ernest DWELT AT 44-rue-Jacob FROM 1921-12-22 UNTIL 1922-01-09; hemingway-ernest DWELT AT 6-rue-Férou FROM 1927-09-19 UNTIL 1931-05-28 

solution: hemingway-ernest DWELT AT 6-rue-Férou FROM 1927-09-19 UNTIL 1931-05-28 




'hemingway-ernest DWELT AT 6-rue-Férou FROM 1927-09-19 UNTIL 1931-05-28'

#### TODO: We should test this more!!

## ... Book Dwellings?

In [210]:
def get_book_dwelling_id(row):
    return f'{row.event_id} WHILE {row.dwelling_id}'

@cache
def get_book_dwellings_df(sep='; '):
    df_borrows = get_borrow_events_df().reset_index()
    
    
    odf = pd.DataFrame([
        {**dict(row), **{'dwelling_id':dwelling_id}}
        for i, row in df_borrows.iterrows()
        for dwelling_id in row.dwelling_ids.split(sep)
        if row.dwelling_ids and dwelling_id
    ])
    
    odf['book_dwelling_id'] = odf.apply(get_book_dwelling_id, axis=1)
    
    return odf.set_index('book_dwelling_id')

In [211]:
get_book_dwellings_df()

,event_id,member_id,book_id,start_date,end_date,borrow_status,borrow_duration_days,source_type,source_citation,source_manifest,source_image,dwelling_ids,dwelling_id
book_dwelling_id,,,,,,,,,,,,,
lanux-eyre-de BORROWS woolf-night-day FROM 1923 TO 1923-11-07 WHILE lanux-eyre-de DWELT AT 126-boulevard-du-Montparnasse,lanux-eyre-de BORROWS woolf-night-day FROM 192...,lanux-eyre-de,woolf-night-day,1923,1923-11-07,Returned,,Lending Library Card,"Sylvia Beach, Eyre de Lanux Lending Library Ca...",https://figgy.princeton.edu/concern/scanned_re...,https://iiif.princeton.edu/loris/figgy_prod/c5...,lanux-eyre-de DWELT AT 126-boulevard-du-Montpa...,lanux-eyre-de DWELT AT 126-boulevard-du-Montpa...
lanux-eyre-de BORROWS woolf-night-day FROM 1923 TO 1923-11-07 WHILE lanux-eyre-de DWELT AT 17-rue-Visconti,lanux-eyre-de BORROWS woolf-night-day FROM 192...,lanux-eyre-de,woolf-night-day,1923,1923-11-07,Returned,,Lending Library Card,"Sylvia Beach, Eyre de Lanux Lending Library Ca...",https://figgy.princeton.edu/concern/scanned_re...,https://iiif.princeton.edu/loris/figgy_prod/c5...,lanux-eyre-de DWELT AT 126-boulevard-du-Montpa...,lanux-eyre-de DWELT AT 17-rue-Visconti
tery BORROWS james-joyce FROM 1924 TO 1924-10-22 WHILE tery DWELT AT 4-rue-de-l'Hôtel-Colbert,tery BORROWS james-joyce FROM 1924 TO 1924-10-22,tery,james-joyce,1924,1924-10-22,Returned,,Lending Library Card,"Sylvia Beach, Mlle Simone Tery Lending Library...",https://figgy.princeton.edu/concern/scanned_re...,https://iiif.princeton.edu/loris/figgy_prod/c2...,tery DWELT AT 4-rue-de-l'Hôtel-Colbert,tery DWELT AT 4-rue-de-l'Hôtel-Colbert
tery BORROWS freeman-portrait-george-moore FROM 1924 TO 1924-10-22 WHILE tery DWELT AT 4-rue-de-l'Hôtel-Colbert,tery BORROWS freeman-portrait-george-moore FRO...,tery,freeman-portrait-george-moore,1924,1924-10-22,Returned,,Lending Library Card,"Sylvia Beach, Mlle Simone Tery Lending Library...",https://figgy.princeton.edu/concern/scanned_re...,https://iiif.princeton.edu/loris/figgy_prod/c2...,tery DWELT AT 4-rue-de-l'Hôtel-Colbert,tery DWELT AT 4-rue-de-l'Hôtel-Colbert
macleish-ada BORROWS stern-tents-israel FROM 1926 TO 1926-01-14 WHILE macleish-ada DWELT AT 10-parc-de-Montretout,macleish-ada BORROWS stern-tents-israel FROM 1...,macleish-ada,stern-tents-israel,1926,1926-01-14,Returned,,Lending Library Card,"Sylvia Beach, Ada and Archibald MacLeish Lendi...",https://figgy.princeton.edu/concern/scanned_re...,https://iiif.princeton.edu/loris/figgy_prod/22...,macleish-ada DWELT AT 10-parc-de-Montretout; m...,macleish-ada DWELT AT 10-parc-de-Montretout
...,...,...,...,...,...,...,...,...,...,...,...,...,...
hommel-bernard BORROWS heine-poems WHILE hommel-bernard DWELT AT 12-rue-de-l'Odéon,hommel-bernard BORROWS heine-poems,hommel-bernard,heine-poems,,,Returned,,Lending Library Card,"Sylvia Beach, Bernard Hommel Lending Library C...",https://figgy.princeton.edu/concern/scanned_re...,https://iiif.princeton.edu/loris/figgy_prod/e4...,hommel-bernard DWELT AT 12-rue-de-l'Odéon,hommel-bernard DWELT AT 12-rue-de-l'Odéon
hommel-bernard BORROWS cowley-exiles-return-narrative WHILE hommel-bernard DWELT AT 12-rue-de-l'Odéon,hommel-bernard BORROWS cowley-exiles-return-na...,hommel-bernard,cowley-exiles-return-narrative,,,Returned,,Lending Library Card,"Sylvia Beach, Bernard Hommel Lending Library C...",https://figgy.princeton.edu/concern/scanned_re...,https://iiif.princeton.edu/loris/figgy_prod/e4...,hommel-bernard DWELT AT 12-rue-de-l'Odéon,hommel-bernard DWELT AT 12-rue-de-l'Odéon
hommel-bernard BORROWS wilson-axels-castle-study WHILE hommel-bernard DWELT AT 12-rue-de-l'Odéon,hommel-bernard BORROWS wilson-axels-castle-study,hommel-bernard,wilson-axels-castle-study,,,Returned,,Lending Library Card,"Sylvia Beach, Bernard Hommel Lending Library C...",https://figgy.princeton.edu/concern/scanned_re...,https://iiif.princeton.edu/loris/figgy_prod/e4...,hommel-bernard DWELT AT 12-rue-de-l'Odéon,hommel-bernard DWELT AT 12-rue-de-l'Odéon
